In [1]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import create_map, lit

In [2]:
airlines_schema = StructType([
    StructField("iata_code", StringType(), True),
    StructField("airline", StringType(), True)
])

In [3]:
# df = spark.read.csv("/Volumes/unicargo_dev/00_raw/source_unicargo_data/airlines.csv", schema=schema)

In [ ]:
# Create widgets (required for ADF)
dbutils.widgets.text("pipeline_id", "")
dbutils.widgets.text("run_id", "")
dbutils.widgets.text("task_id", "")
dbutils.widgets.text("processed_timestamp", "")
dbutils.widgets.text("catalog", "unikargo_dev") # Requires an ADF variable for ADF runs

In [ ]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

In [ ]:
df = (spark.read
    .schema(airlines_schema)    # your predefined schema
    .option("header", "true")   # use the header row for column names only
    # .option("inferSchema", "false")  # do not infer schema from CSV
    # .csv(f"/Volumes/{catalog}/00_raw/source_unicargo_data/airlines.csv")
    .csv("abfss://medallion@adlsunikarrgodev.dfs.core.windows.net/raw/volumes/airlines.csv") # added for adf
)

In [ ]:
df = df.withColumn("metadata",
                   create_map(
                       lit("pipeline_id"), lit(pipeline_id),
                       lit("run_id"), lit(run_id),
                       lit("task_id"), lit(task_id),
                       lit("processed_timestamp"), lit(processed_timestamp),
                   ))

In [6]:
# df.show(5)

In [ ]:
df.write.\
mode("overwrite").\
option("overwriteSchema", "true").\
saveAsTable(f"`{catalog}`.`01_bronze`.`unikargo_airlines_bronze`") 
# Python f-string expands to: unikargo_dev.01_bronze.unikargo_airlines_bronze.
# Python f-string expands to: unikargo_dev.01_bronze.unikargo_airlines_bronze

# The SQL parser thinks 01_bronze is a number, because it starts with a digit.

# SQL identifiers cannot start with a number unless you quote them. 
# 
# schemas now quoted for SQ